In [29]:
import os, sys, json
from xml.etree.ElementTree import parse
from urllib.parse import urlencode, unquote, quote_plus
import xmltodict
import pandas as pd

if sys.version_info[0]==3:
    from urllib.request import urlopen
else:
    from urllib import urlopen


BASE_DIR = './'

# Secret File Load
secret_file = os.path.join(BASE_DIR,'secret.json')
with open(secret_file) as f:
    secrets = json.loads(f.read())

KEY = unquote(secrets['Enecoding'])

**setting**
bulidingtype = 0 : total, 1: apt, 3 : ali, 7 : sgl
regio = city(36000 : sejong, A1000 : 전국, A2000 : 수도권, 11000 : seoul)
startmonth/endmonth = yyyymm

In [319]:
from itertools import product

b_type = ['0','1','3','7']
b_region = ['36000', 'A1000', 'A2000', '11000']
list_com = list(product(b_region,b_type))

In [320]:
def call_open_api(region,_type) : 

    URL = 'http://openapi.reb.or.kr/OpenAPI_ToolInstallPackage/service/rest/TradePrcIndexSvc/getTradePrcIndex'
    queryParams = '?' + urlencode(
        {
            quote_plus('serviceKey') : KEY,
            quote_plus('startmonth') : '201701',
            quote_plus('endmonth') : '202102',
            quote_plus('region') : region,
            quote_plus('buildingtype') : _type,

        }
    )

    response = urlopen(URL + queryParams).read()
    decode_data = response.decode('utf-8')
    
    xml_parse =  xmltodict.parse(decode_data)
    xml_dict = json.loads(json.dumps(xml_parse))

    return xml_dict

# cc = xmltodict.parse(tree) # return collections.OrderedDict
# dd = json.loads(json.dumps(cc)) # return dict
# animals = dd['animals']['animal']
# print(animals) # 결과를 출력한다

In [340]:
def writeDatafram(region, _type) :

    new_data = call_open_api(region, _type)
    new_data = new_data['response']['body']['item']
   
    df = pd.Series(new_data)
    
    return df

In [420]:
api_data = pd.DataFrame()
for i in list_com :

    (region, _type) = i
    df = writeDatafram(region, _type)

    api_data = api_data.append(df, ignore_index=True)

api_data['name'] = api_data['aptTypeNm'] + api_data['regionCd'] + api_data['regionNm']
api_data = api_data.set_index('name')
api_data

,aptTypeNm,regionCd,regionNm,rsRow
name,,,,
전체36000세종,전체,36000,세종,"201701,96.0|201702,96.0|201703,96.0|201704,96...."
아파트36000세종,아파트,36000,세종,"201701,95.9|201702,95.8|201703,95.7|201704,95...."
연립/다세대36000세종,연립/다세대,36000,세종,"201701,99.4|201702,99.4|201703,99.3|201704,99...."
단독36000세종,단독,36000,세종,"201701,96.0|201702,96.2|201703,96.5|201704,96...."
전체A1000전국,전체,A1000,전국,"201701,98.7|201702,98.7|201703,98.7|201704,98...."
아파트A1000전국,아파트,A1000,전국,"201701,99.0|201702,99.0|201703,99.0|201704,99...."
연립/다세대A1000전국,연립/다세대,A1000,전국,"201701,99.0|201702,99.0|201703,99.1|201704,99...."
단독A1000전국,단독,A1000,전국,"201701,97.6|201702,97.7|201703,97.8|201704,98...."
전체A2000수도권,전체,A2000,수도권,"201701,97.9|201702,98.0|201703,98.0|201704,98...."


In [422]:
rate = api_data.rsRow.str.split('|',expand=True)

for i in range(50) :

    col_name = rate[i].str.split(',').get(1)[0]
    rate[i] = rate[i].str.split(',').str[1]

    rate = rate.rename(columns={ i : col_name})


In [424]:
api_data_final = pd.merge(api_data, rate, left_index=True,right_index=True, how='outer')
api_data_final = api_data_final.drop('rsRow', axis=1)
api_data_final = api_data_final.reset_index(drop=True)
api_data_final

,aptTypeNm,regionCd,regionNm,201701,201702,201703,201704,201705,201706,201707,...,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102
0,전체,36000,세종,96.0,96.0,96.0,96.2,96.8,98.5,99.1,...,111.7,114.0,120.2,129.4,134.4,136.3,137.6,138.8,140.0,141.2
1,아파트,36000,세종,95.9,95.8,95.7,95.9,96.6,98.7,99.4,...,111.1,113.9,121.4,132.5,138.5,140.6,141.9,143.2,144.6,146.0
2,연립/다세대,36000,세종,99.4,99.4,99.3,99.3,99.4,99.5,99.6,...,97.6,97.8,97.9,98.2,98.3,98.9,99.4,99.9,100.1,100.4
3,단독,36000,세종,96.0,96.2,96.5,96.7,97.1,97.6,98.2,...,113.6,114.0,114.6,116.3,117.5,118.8,119.9,121.0,121.8,122.4
4,전체,A1000,전국,98.7,98.7,98.7,98.8,99.0,99.2,99.4,...,102.4,102.9,103.5,104.0,104.4,104.8,105.3,106.3,107.1,108.1
5,아파트,A1000,전국,99.0,99.0,99.0,99.1,99.2,99.4,99.6,...,100.9,101.5,102.4,103.0,103.6,104.0,104.8,106.2,107.4,108.9
6,연립/다세대,A1000,전국,99.0,99.0,99.1,99.2,99.3,99.4,99.6,...,100.1,100.2,100.3,100.5,100.6,100.7,100.9,101.0,101.3,101.5
7,단독,A1000,전국,97.6,97.7,97.8,98.0,98.3,98.5,98.8,...,107.6,107.8,108.1,108.3,108.6,108.8,109.1,109.4,109.6,109.9
8,전체,A2000,수도권,97.9,98.0,98.0,98.2,98.4,98.7,99.0,...,106.8,107.3,108.2,108.7,109.2,109.5,110.1,110.8,111.7,113.0
9,아파트,A2000,수도권,97.7,97.7,97.8,97.9,98.2,98.6,98.9,...,107.7,108.4,109.7,110.5,111.1,111.5,112.2,113.3,114.5,116.5


In [425]:
api_data_final.to_csv('api_tradePrc.csv')